In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import sys
import random

In [2]:
data_pairs = pd.read_csv("./data/train.csv")
data_pairs = data_pairs.drop("id", axis=1)
data_pairs.head()

,qid1,qid2,question1,question2,is_duplicate
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
data_questions = pd.read_csv("./data/data_questions.csv")
data_questions.head()

,qid,question
0,1,What is the step by step guide to invest in sh...
1,2,What is the step by step guide to invest in sh...
2,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
3,4,What would happen if the Indian government sto...
4,5,How can I increase the speed of my internet co...


In [4]:
qids = pd.unique(data_pairs[["qid1", "qid2"]].values.ravel())
print(f"broj parova: {len(data_pairs)}")
print(f"broj pitanja: {len(qids)}")

broj parova: 404290
broj pitanja: 537933


In [5]:
sum(data_pairs["is_duplicate"] == 1)

149263

# Analiza tranzitivnih ovisnosti

moze se na dataset gledati kao na graf
pitanje je vrh, a similarity == 1 je brid
onda je augmentacija dodavnje parova svakog drugog susjeda, treceg, ...
maksimalna augmentacija za similar primjere bi bila da nedemo sve komponente grafa i dodamo sve parove u svakoj komponenti

jos mozemo razmotrit drugi graf kojem je brid similarity == 0 i dodavi tranzitivne parove koji nisu similar
mozda bi imalo smisla da dodajemo jedan ne similar primjer za svaki similar ili mozda dva. Da odrzimo dataset balansiram

In [6]:
# question_id -> question_id
graph_similar = defaultdict(lambda: [])
original_similar_pairs = set()
for _, (qid1, qid2, question1, question2, is_duplicate) in data_pairs.iterrows():
    if not is_duplicate: continue

    graph_similar[qid1].append(qid2)
    graph_similar[qid2].append(qid1)

    lower_id = min(qid1, qid2)
    higher_id = max(qid1, qid2)
    original_similar_pairs.add((lower_id, higher_id))

In [7]:
second_neighbors = set()
for qid, neighbours in graph_similar.items():
    for neighbour in neighbours:
        for second_neighbor in graph_similar[neighbour]:
            if second_neighbor == qid:
                continue

            lower_id = min(qid, second_neighbor)
            higher_id = max(qid, second_neighbor)
            if (lower_id, higher_id) not in original_similar_pairs:
                second_neighbors.add((lower_id, higher_id))

In [8]:
second_neighbors = list(second_neighbors)
second_neighbors.sort()
print(f"second neighbours size: {len(second_neighbors)}")
second_neighbors

second neighbours size: 55490


[(31, 1100),
 (31, 2066),
 (31, 2067),
 (31, 6079),
 (31, 6938),
 (31, 7751),
 (31, 7752),
 (31, 8577),
 (31, 8578),
 (31, 11434),
 (31, 16474),
 (31, 17171),
 (31, 24203),
 (31, 24204),
 (31, 24960),
 (31, 32509),
 (31, 32957),
 (31, 36835),
 (31, 36836),
 (31, 37617),
 (31, 38502),
 (31, 46127),
 (31, 51528),
 (31, 53535),
 (31, 64848),
 (31, 77287),
 (31, 83195),
 (31, 93146),
 (31, 106091),
 (31, 120229),
 (31, 132922),
 (31, 140488),
 (31, 167907),
 (31, 218253),
 (31, 221900),
 (31, 258354),
 (31, 333813),
 (31, 528360),
 (32, 6079),
 (32, 6080),
 (32, 7751),
 (32, 7752),
 (32, 8577),
 (32, 8578),
 (32, 17171),
 (32, 24203),
 (32, 24204),
 (32, 24960),
 (32, 32509),
 (32, 32957),
 (32, 36835),
 (32, 36836),
 (32, 38502),
 (32, 44686),
 (32, 51528),
 (32, 64848),
 (32, 77287),
 (32, 81385),
 (32, 83195),
 (32, 93146),
 (32, 100431),
 (32, 106091),
 (32, 120229),
 (32, 140488),
 (32, 140581),
 (32, 167907),
 (32, 218253),
 (32, 249733),
 (32, 258354),
 (32, 333813),
 (37, 2374),
 (

In [9]:
for i in range(8):
    qid1, qid2 = second_neighbors[i]
    first_question = data_questions.iloc[qid1 - 1].question
    second_question = data_questions.iloc[qid2 - 1].question
    print(first_question)
    print(second_question)
    print()

What would a Trump presidency mean for current international master’s students on an F1 visa?
How will Trump's presidency affect Indian students who are planning to do a PhD in the US?

What would a Trump presidency mean for current international master’s students on an F1 visa?
I am an Indian, planning to go to US for MS (a STEM course) this January. If Trump wins, how will that affect my future in US?

What would a Trump presidency mean for current international master’s students on an F1 visa?
How is Trump becoming the president affect the Indians applying for an MS in the US (Mech)?

What would a Trump presidency mean for current international master’s students on an F1 visa?
Now that Donald Trump is President, will international students stop coming to US universities?

What would a Trump presidency mean for current international master’s students on an F1 visa?
How might Trump affect the status of foreign students at top universities in the US?

What would a Trump presidency mean

In [10]:
for i in range(8):
    qid1, qid2 = second_neighbors[i * 100]
    first_question = data_questions.iloc[qid1 - 1].question
    second_question = data_questions.iloc[qid2 - 1].question
    print(first_question)
    print(second_question)
    print()

What would a Trump presidency mean for current international master’s students on an F1 visa?
How will Trump's presidency affect Indian students who are planning to do a PhD in the US?

Why are so many Quora users posting questions that are readily answered on Google?
Why do people ask questions here in Quora instead of just googling?

I can't remember my Gmail password or my recovery email. How can I recover my e-mail?
How do you rest your rescue password if you don't remember your answers to the sequrity questions?

How can I learn to speak English fluently?
How could I be fluent in English?

Will there really be any war between India and Pakistan over the Uri attack? What will be its effects?
Is there a chance for India v/s Pakistan War?

What are the effects of demonitization of 500 and 1000 rupees notes on real estate sector?
What will be the effect on real estate sector after demonetization of currency in India?

What exactly is GST bill and how exactly will it affect the common 

# provjera jel procurio neki primjer koji je vec bio u datasetu

In [11]:
original_similar_pairs = set()
for _, (qid1, qid2, question1, question2, is_duplicate) in data_pairs.iterrows():
    if is_duplicate:
        lower_id = min(qid1, qid2)
        higher_id = max(qid1, qid2)
        original_similar_pairs.add((lower_id, higher_id))

augmented_paris = set(second_neighbors)

In [12]:
print(data_pairs.is_duplicate.sum())
print(len(original_similar_pairs))
print(len(augmented_paris))

149263
149263
55490


In [13]:
intersection = original_similar_pairs.intersection(augmented_paris)
print(len(intersection))

0


# broj komponenti

In [14]:
labels = defaultdict(lambda : 0)
component_count = defaultdict(lambda: 0)

def visit(node, level):
    stack = [node]
    while len(stack) > 0:
        v = stack.pop()
        if labels[v - 1] != 0: continue
        labels[v - 1] = level
        component_count[level] += 1
        stack += graph_similar[v]

level = 0
for node in graph_similar:
    if labels[node - 1] == 0:
        level += 1
        visit(node, level)

print(level)

60460


In [15]:
component_counts = list(component_count.values())
component_counts.sort()
print(component_counts)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

# rastav na tranzicije po koracima

In [16]:
transitive_neighbours = defaultdict(lambda : 200)

def find_neighbours(node):
    depth = 1
    visited = {node}
    queue = [node]
    while len(queue):
        next_neighbours = [u for v in queue for u in graph_similar[v] if u not in visited]
        for neighbour in next_neighbours:
            visited.add(neighbour)
            lower_id, higher_id = min(node, neighbour), max(node, neighbour)
            transitive_neighbours[(lower_id, higher_id)] = depth

        depth += 1
        queue = list(set(next_neighbours))

for node in graph_similar:
    find_neighbours(node)

In [17]:
max_steps = max(transitive_neighbours.values())
transitive_pairs = []
for step in range(max_steps + 1):
    transitive_pairs.append([])

for pair, depth in transitive_neighbours.items():
    transitive_pairs[depth].append(pair)

In [25]:
counts = [len(pairs) for pairs in transitive_pairs]
print(counts)
print(sum(counts[2:]))

[0, 149263, 55490, 17920, 4279, 1217, 288, 74, 16, 1]
79285


# dodavanje negativnih primjera

In [19]:
n_questions = len(data_questions)

def generate_random_pair():
    random_first = random.randint(1, n_questions)
    random_second = random.randint(1, n_questions)
    return min(random_first, random_second), max(random_first, random_second)

def generate_negative_examples(n):
    result = set()
    while len(result) < n:
        pair = generate_random_pair()
        while pair in original_similar_pairs:
            pair = generate_random_pair()
        result.add(pair)

    return list(result)

In [20]:
negative_examples = generate_negative_examples(1000000)

In [21]:
set1 = original_similar_pairs
set2 = negative_examples
print(len(set1))
print(len(set2))
print(len(set1.intersection(set2)))

149263
1000000
0


In [22]:
for i in range(8):
    qid1, qid2 = negative_examples[i]
    first_question = data_questions.iloc[qid1 - 1].question
    second_question = data_questions.iloc[qid2 - 1].question
    print(first_question)
    print(second_question)
    print()

What individuals and events in history are a source of pride for Angola?
How did you discover masturbation?

Given [math]A[/math], What is the general formula of [math]A^n[/math]? If [math]A=\begin{pmatrix} -2 &4 \\ -5& 7 \end{pmatrix}[/math], what is the [math]A^n[/math]?
What are some of the most under-appreciated things in life?

On the whole, is gentrification really wrong?
What are the ways for a stupid person to earn money online?

Arrear fee in SRM?
I have $30000. What is the best way to invest this?

How can I get a hacker?
What can be other career/job profiles for someone with a degree in computer engineering?

Which programming language will be in demand?
What is your review of Nike+?

Why do I get blackheads on my nose?
How do I improve logical programming skills?

Who makes the First Round Holiday videos?
How was the relationship between Steve Wozniak and Steve Jobs?

